![](https://www.snowflake.com/wp-content/themes/snowflake/assets/img/brand-guidelines/logo-sno-blue-example.svg)

# Build, Deploy and Monitor your Model in Snowflake

In this demo we will be showcasing how a complete model life cycle looks like in Snowflake. We will be using the following capabilities in Snowflake,

* Snowflake ML Python SDK
* Model Registry
* ML Observability
* Alerts
* Drift Monitoring

![](https://drive.google.com/file/d/1jWryVEAjyetHMRgTTMo_bnx_BZRdeNuC/view?usp=sharing)

>**Use case:** A financial institution has been dealing with loss of customers to competition. They want to understand the likelihood of each of their customer's churning so that they can take necessary action for users with high probablity of churning.Over a period of time there is a new trend seen in customer churn which needs to be actionized immediately to gain competitive advantage. The financial institution leverages the ML Observability dashboard to view the metrics and accuracy factors. It can take proactive action for retraining the model and is also able to compare different model version by monitoring the model performance.

### **Features**

* **CREDITSCORE:** Credit score of the customer based on their historical credit behavior and management  
* **GEOGRAPHY:** Country of residence
* **GENDER:** Gender of the customer
* **AGE:** Age of the customer
* **TENURE:** Duration in years that they have been a customer
* **BALANCE:** Current balance of their bankaccount
* **NUMOFPRODUCTS:** Number of products purchased from the bank
* **HASCRCARD:** Does the customer have a credit card? - 1 if they do, 0 if they don't
* **ISACTIVEMEMBER:** Has the customer used their bank account in the last 3 months? - 1 if they did, 0 if they didn't
* **ESTIMATEDSALARY:** Estimated salary of the customer
* **DEBTTOINCOME:** Debt to income ratio

### **Model**

We will build a classification model using XGBoost framework with Snowflake ML API and log the model to registry. Along the way we will create model monitors and view the model performance in Snowsight dashboard.

In [58]:
# Import python packages
import streamlit as st
import pandas as pd

from snowflake.snowpark.context import get_active_session
session = get_active_session()
from datetime import datetime, timedelta
from snowflake.ml.registry import Registry
import joblib
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake.ml.modeling.preprocessing as pp
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.snowpark.types import StringType, IntegerType
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import col, current_date, dateadd, random, floor,current_date, datediff

session.query_tag = {"origin":"sf_sit-is", "name":"mlops_customerchurn", "version":{"major":1, "minor":0}}

import snowflake.snowpark.functions as F
from IPython.display import Markdown, display

solution_prefix = session.get_current_warehouse()
solution_prefix

### Load synthetic data from the data_stage into a Snowflake table using a COPY INTO command.

In [66]:
-- Create csv format
CREATE FILE FORMAT IF NOT EXISTS CSVFORMAT 
    SKIP_HEADER = 1 
    TYPE = 'CSV';
    
CREATE OR REPLACE STAGE data_stage
    FILE_FORMAT = (TYPE = 'CSV') 
    URL = 's3://sfquickstarts/sfguide_getting_started_with_ml_observability_in_snowflake/mlops_customerchurn.csv';
    
-- Inspect content of stage
LS @data_stage;


Total exited customers: 1714 (Target: ~2000)
   CustomerId  Surname  CreditScore Geography  Gender  Age  Tenure    Balance  \
0           1    Johns          402    France    Male   55       9   91944.03   
1           2  Schultz          735     Spain    Male   59       8  126536.56   
2           3    Jones          570     Spain    Male   54       7  191357.66   
3           4    Baker          406    France  Female   73       3  125263.00   
4           5  Aguirre          371     Spain    Male   88       9  195626.75   

   NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  Exited  \
0              1          1               1         36899.18       0   
1              2          0               0         33120.74       0   
2              2          1               1         34751.09       1   
3              4          0               0        169844.77       0   
4              4          0               1         13787.72       0   

  TransactionTimestamp  debttoincom

### Read a CSV file using Snowpark from a stage in Snowflake into a DataFrame. 

In [67]:
spdf = session.read.options({"field_delimiter": ",",
                                    "field_optionally_enclosed_by": '"',
                                    "infer_schema": True,
                                    "parse_header": True}).csv("@data_stage")



In [ ]:
from snowflake.snowpark.types import DecimalType, FloatType, DoubleType

# Get schema of the DataFrame
schema = spdf.schema.fields

# Identify columns that are of type NUMBER (DecimalType)
num_columns = [col.name for col in schema if isinstance(col.datatype, DecimalType)]

# Convert columns to FLOAT
for col in num_columns:
    spdf = spdf.with_column(col, spdf[col].cast(DoubleType()))



In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, current_date, dateadd, to_date,lit

# Step 1: Get today's date
todays_date = datetime.now()

latest_date = max(spdf.select('TRANSACTIONTIMESTAMP').collect())[0]

# Step 3: Calculate the difference in days
diff_days = (todays_date - latest_date).days - 1

df = spdf.with_column(
    "TRANSACTIONTIMESTAMP", 
    dateadd("day", lit(diff_days), col("TRANSACTIONTIMESTAMP"))
)


df = df.with_column(
    "CREDITSCORE", col("CREDITSCORE").cast("float")
)
df = df.with_column(
    "PREDICTED_CHURN", F.lit(9999)
)
df.show()

df.write.mode("overwrite").save_as_table("CUSTOMERS")


In [75]:
spdf= df.drop('ROWNUMBER')



--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMERID"  |"CREDITSCORE"  |"GEOGRAPHY"  |"GENDER"  |"AGE"  |"TENURE"  |"BALANCE"  |"NUMOFPRODUCTS"  |"HASCRCARD"  |"ISACTIVEMEMBER"  |"ESTIMATEDSALARY"  |"EXITED"  |"TRANSACTIONTIMESTAMP"  |"DEBTTOINCOME"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1             |402            |France       |Male      |55     |9         |91944.03   |1                |1            |1                 |36899.18           |0         |2022-01-09 14:08:54     |23              |
|2             |735            |Spain        |Male      |59     |8         |126536.56  |2                |0            |0                 |33120.74 

#### Define a preprocessing pipeline using Pipeline with two steps: Ordinal Encoding for categorical columns and Min-Max Scaling for numerical columns. It then splits the data into training and testing sets, applies the preprocessing steps to the training data, and saves the pipeline as a joblib file (preprocessing_pipeline.joblib) .

In [ ]:
num_cols = ['ESTIMATEDSALARY', 'BALANCE', 'CREDITSCORE','AGE','TENURE','DEBTTOINCOME']
output_cols=['EstimatedSalary_SS', 'Balance_SS', 'CreditScore_SS','Age_SS','Tenure_SS','debttoincome_SS']

cat_cols = ['HasCrCard', 'IsActiveMember', 'Geography','Gender', 'NumOfProducts']
string_columns = ['GEOGRAPHY', 'GENDER']
string_columns_oe = ['GEOGRAPHY_oe', 'GENDER_oe']
preprocessing_pipeline = Pipeline(
    steps=[
            (
                "OE",
                pp.OrdinalEncoder(
                    input_cols=string_columns,
                    output_cols=string_columns_oe,
                    drop_input_cols= False,
                )
                
            ),
            (
                "MMS",
                pp.MinMaxScaler(
                    clip=True,
                    input_cols=num_cols,
                    output_cols=output_cols,
                    drop_input_cols= False,
                )
            )
    ]
)

PIPELINE_FILE = '/tmp/preprocessing_pipeline.joblib'
joblib.dump(preprocessing_pipeline, PIPELINE_FILE) # We are just pickling it locally first
training, testing = spdf.random_split(weights=[0.8, 0.2], seed=111)
training_spdf = preprocessing_pipeline.fit(training).transform(training)
testing_spdf=preprocessing_pipeline.fit(testing).transform(testing)

#### Store the pipeline file in a stage

In [ ]:
session.sql("CREATE or replace stage ML_STAGE").collect()
session.file.put(PIPELINE_FILE, "@ML_STAGE", overwrite=True)

In [ ]:
ls @ML_STAGE

## Build the XGBClassifier model and train using the training data

In [83]:
num_cols = ['EstimatedSalary', 'Balance', 'CreditScore','Age','Tenure','debttoincome']

cat_cols = ['HasCrCard', 'IsActiveMember', 'GEOGRAPHY','GENDER', 'NumOfProducts']
Target = ["EXITED"]

feature_names_input = [c for c in training_spdf.columns if c not in ["EXITED", "TRANSACTIONTIMESTAMP", "CUSTOMERID","ESTIMATEDSALARY", "BALANCE", "CREDITSCORE","AGE","TENURE","DEBTTOINCOME","GEOGRAPHY","GENDER","PREDICTED_CHURN"]]


training_spdf = training_spdf.with_column(
    "CREDITSCORE_SS", col("CREDITSCORE_SS").cast("float")
)
output_label = ["PREDICTED_CHURN"]
# Initialize a XGBClassifier object with input, label, and output column names
model = XGBClassifier(
    input_cols=feature_names_input,
    label_cols=Target,
    output_cols=output_label
    
)

# Train the classifier model using the training set
_ = model.fit(training_spdf)



### Initalize Snowflake Model Registry

Log and manage the trained machine learning model in Snowflake.

Notice the task=type_hints.Task.TABULAR_BINARY_CLASSIFICATION: Specifies that this is a binary classification task (predicting churn: Yes/No).

In [ ]:
from snowflake.ml.registry import Registry
from snowflake.ml.model import type_hints

reg = Registry(session=session)

MODEL_NAME = "QS_CustomerChurn_classifier"
MODEL_VERSION = "v1"

mv = reg.log_model(model,
                   model_name=MODEL_NAME,
                   version_name=MODEL_VERSION,
                   options={'relax_version': True},
                   task=type_hints.Task.TABULAR_BINARY_CLASSIFICATION)
reg.show_models()


## Ongoing inference
The inference function performs predictions using a pre-trained machine learning model in Snowflake. This function uses the preprocessing pipeline created earlier to ensure data is transformed consistently.
✅ Runs predictions using a registered model version.
✅ Updates predictions directly into the Snowflake table.
✅ Efficiently handles batch inference with SQL updates.

In [ ]:
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake.ml.modeling.preprocessing as pp
import snowflake.snowpark.functions as F

def inference(table_name, modelname, modelversion) -> str:
    reg = Registry(session=session)
    m = reg.get_model(modelname)
    mv = m.version(modelversion)
    
    # Load preprocessing pipeline from a file
    session.file.get('@ML_STAGE/preprocessing_pipeline.joblib.gz', '/tmp')
    pipeline_file = '/tmp/preprocessing_pipeline.joblib.gz'
    
    
    preprocessing_pipeline = joblib.load(pipeline_file)
    
    df = session.table(table_name)
    
    # Apply preprocessing
    testing_spdf = preprocessing_pipeline.fit(df).transform(df)
    testing_spdf = testing_spdf.with_column(
    "CREDITSCORE_SS", col("CREDITSCORE_SS").cast("float")
)
    # Perform prediction
    results = mv.run(testing_spdf, function_name="predict")
    results =results.drop("CREDITSCORE_SS", "BALANCE_SS", "DEBTTOINCOME_SS", "TENURE_SS", "AGE_SS", "ESTIMATEDSALARY_SS", "GENDER_OE", "GEOGRAPHY_OE")
    #results.write.save_as_table("customer_churn", mode="overwrite")
    results.create_or_replace_temp_view("results_temp")
    update_statement = f"""
    UPDATE {table_name} t
    SET PREDICTED_CHURN = r.PREDICTED_CHURN
    FROM results_temp r
    WHERE t.CUSTOMERID = r.CUSTOMERID
    AND t.TRANSACTIONTIMESTAMP=r.TRANSACTIONTIMESTAMP;
"""

    # Execute the merge statement
    session.sql(update_statement).collect()
        
    return "Success"


Execute the trained model on the testing_spdf DataFrame using mv.run().
function_name="predict" specifies that the function to be used for inference is "predict", 
The output is a DataFrame containing predictions.

In [ ]:
testing_spdf = testing_spdf.with_column(
    "CREDITSCORE_SS", col("CREDITSCORE_SS").cast("float")
)
# Perform prediction
results = mv.run(testing_spdf, function_name="predict")
results

## DATA DRIFT AND OBSERVABILITY IN ML OBSERVABILITY DASHBOARD

Now lets see how a data drift can be monitored and proactive action could help the financial firm to prevent customer churn

In [ ]:
-- Create csv format
CREATE FILE FORMAT IF NOT EXISTS CSVFORMAT 
    SKIP_HEADER = 1 
    TYPE = 'CSV';
    
CREATE OR REPLACE STAGE data_stage
    FILE_FORMAT = (TYPE = 'CSV') 
    URL = 's3://sfquickstarts/sfguide_getting_started_with_ml_observability_in_snowflake/CUSTOMERS_DRIFTED.csv';
    
-- Inspect content of stage
LS @data_stage;




The data in CUSTOMERS_DRIFTED file contains new customer trends leading to poor accuracy from inference using the v1 version of the model.

In [ ]:
spdf = session.read.options({"field_delimiter": ",",
                                    "field_optionally_enclosed_by": '"',
                                    "infer_schema": True,
                                    "parse_header": True}).csv("@data_stage")

from snowflake.snowpark.types import DecimalType, FloatType

# Get schema of the DataFrame
schema = spdf.schema.fields

# Identify columns that are of type NUMBER (DecimalType)
num_columns = [col.name for col in schema if isinstance(col.datatype, DecimalType)]

# Convert columns to FLOAT
for col in num_columns:
    spdf = spdf.with_column(col, spdf[col].cast(FloatType()))


from snowflake.snowpark.functions import col, current_date, dateadd, to_date, lit,to_timestamp
from datetime import datetime

# Step 1: Get today's date
todays_date = datetime.now()

# Ensure TRANSACTIONTIMESTAMP is stored as a string first
spdf = spdf.with_column("TRANSACTIONTIMESTAMP", col("TRANSACTIONTIMESTAMP").cast("string"))

# Get the latest date from the dataset
latest_date_str = max(spdf.select('TRANSACTIONTIMESTAMP').collect())[0]

# Convert latest_date to datetime
latest_date = datetime.strptime(latest_date_str, '%m/%d/%y %H:%M')

# Step 3: Calculate the difference in days
diff_days = (todays_date - latest_date).days - 1

# Apply date adjustment
df = spdf.with_column(
    "TRANSACTIONTIMESTAMP",
    dateadd("day", lit(diff_days), to_timestamp(col("TRANSACTIONTIMESTAMP"), 'MM/DD/YY HH24:MI'))
)

# Cast CREDIT SCORE to float
df = df.with_column("CREDITSCORE", col("CREDITSCORE").cast("float"))

# Add PREDICTED_CHURN column
spdf_drift = df.with_column("PREDICTED_CHURN", lit(9999))

spdf_drift.show()


The financial firm carries the predictions with the drifted data using the model version v1. The drifted data is saved in CUSTOMERS_DRIFTED table. A copy of the same data is saved in the CUSTOMERS_EVAL table to show how that drift was monitored and a new model retrained proactively to avert inaccurate decision making. 

In [ ]:

current_columns = spdf_drift.columns 
new_columns = [col.strip('"') for col in current_columns] 

spdf_drift = spdf_drift.select([spdf_drift[col].alias(new_col) for col, new_col in zip(current_columns, new_columns)])
spdf_drift = spdf_drift.with_column(
    "CREDITSCORE", col("CREDITSCORE").cast("float")
)
spdf_drift.write.mode("overwrite").save_as_table("CUSTOMERS_DRIFTED")
spdf_drift.write.mode("overwrite").save_as_table("CUSTOMERS_EVAL")


# Enable Monitoring
Create a model monitor using the CREATE MODEL MONITOR command. The monitor object automatically refreshes the monitor logs by querying source data and updates the monitoring reports based on the logs. The first one that is commented out shows the pythonic method and the next cell shows how to monitor using SQL

In [ ]:
'''
reg = Registry(session=session,options={"enable_monitoring": True})
modelname='QS_CustomerChurn_classifier'
modelversion='v1'
m = reg.get_model(modelname)
mv = m.version(modelversion)

# Fetch model version that will be monitored
model_version = mv

from snowflake.ml.monitoring.entities.model_monitor_config import ModelMonitorConfig, ModelMonitorSourceConfig
source_config = ModelMonitorSourceConfig(
    source="CUSTOMERS_DRIFTED",
    baseline="CUSTOMERS",
    timestamp_column="TRANSACTIONTIMESTAMP",
    prediction_class_columns=["PREDICTED_CHURN"],
    actual_class_columns=["EXITED"],
    id_columns=["CUSTOMERID"],
)

# Set up config for ModelMonitor.
model_monitor_config = ModelMonitorConfig(
    model_version=model_version,
    model_function_name="predict",
    background_compute_warehouse_name="ml_wh",
)

# Add a new ModelMonitor
model_monitor = reg.add_monitor(
    name=f"CHURN_MODEL_MONITOR", 
    source_config=source_config,
    model_monitor_config=model_monitor_config,
)
model_monitor
'''

## SQL Version for creating the model monitor 

In [ ]:
query = f"""
CREATE OR REPLACE MODEL MONITOR CHURN_MODEL_MONITOR
WITH
    MODEL=QS_CustomerChurn_classifier
    VERSION=v1
    FUNCTION=predict
    SOURCE=CUSTOMERS_DRIFTED
    BASELINE=CUSTOMERS
    TIMESTAMP_COLUMN=TRANSACTIONTIMESTAMP
    PREDICTION_CLASS_COLUMNS=(PREDICTED_CHURN)  
    ACTUAL_CLASS_COLUMNS=(EXITED)
    ID_COLUMNS=(CUSTOMERID)
    WAREHOUSE=ML_WH
    REFRESH_INTERVAL='1 min'
    AGGREGATION_WINDOW='1 day';
"""
session.sql(query).collect()

## Predict the churn for the new customer trends

In [ ]:
status= inference('CUSTOMERS_DRIFTED','QS_CUSTOMERCHURN_CLASSIFIER', 'v1');

Let us check how the churn predictions and the metrics look like. Open the Dashboard by navigating to Studio->Models

Click on your model and choose the monitor that you just added above.Change the date range to "Last 3 months".

In [ ]:
-- Create csv format
CREATE FILE FORMAT IF NOT EXISTS CSVFORMAT 
    SKIP_HEADER = 1 
    TYPE = 'CSV';
    
CREATE OR REPLACE STAGE data_stage
    FILE_FORMAT = (TYPE = 'CSV') 
    URL = 's3://sfquickstarts/sfguide_getting_started_with_ml_observability_in_snowflake/CUSTOMERS_TRAINING.csv';
    
-- Inspect content of stage
LS @data_stage;


## It can be found by data drift and concept drift has significantly impacted the model's performance over time. To maintain accuracy, models require retraining periodically. Below is how this is handled in the customer churn prediction pipeline. Let us now retrain the data on the new trends.

In [ ]:
spdf = session.read.options({"field_delimiter": ",",
                                    "field_optionally_enclosed_by": '"',
                                    "infer_schema": True,
                                    "parse_header": True}).csv("@data_stage")

from snowflake.snowpark.types import DecimalType, FloatType

# Get schema of the DataFrame
schema = spdf.schema.fields

# Identify columns that are of type NUMBER (DecimalType)
num_columns = [col.name for col in schema if isinstance(col.datatype, DecimalType)]

# Convert columns to FLOAT
for col in num_columns:
    spdf = spdf.with_column(col, spdf[col].cast(FloatType()))


from snowflake.snowpark.functions import col, current_date, dateadd, to_date, lit,to_timestamp
from datetime import datetime

# Step 1: Get today's date
todays_date = datetime.now()

# Ensure TRANSACTIONTIMESTAMP is stored as a string first
spdf = spdf.with_column("TRANSACTIONTIMESTAMP", col("TRANSACTIONTIMESTAMP").cast("string"))

# Get the latest date from the dataset
latest_date_str = max(spdf.select('TRANSACTIONTIMESTAMP').collect())[0]

# Convert latest_date to datetime
latest_date = datetime.strptime(latest_date_str, '%m/%d/%y %H:%M')

# Step 3: Calculate the difference in days
diff_days = (todays_date - latest_date).days - 1

# Apply date adjustment
df = spdf.with_column(
    "TRANSACTIONTIMESTAMP",
    dateadd("day", lit(diff_days), to_timestamp(col("TRANSACTIONTIMESTAMP"), 'MM/DD/YY HH24:MI'))
)

# Cast CREDIT SCORE to float
df = df.with_column("CREDITSCORE", col("CREDITSCORE").cast("float"))

# Add PREDICTED_CHURN column
spdf_drift = df.with_column("PREDICTED_CHURN", lit(9999))

spdf_drift.show()

current_columns = spdf_drift.columns 
new_columns = [col.strip('"') for col in current_columns] 

spdf_drift = spdf_drift.select([spdf_drift[col].alias(new_col) for col, new_col in zip(current_columns, new_columns)])
spdf_drift = spdf_drift.with_column(
    "CREDITSCORE", col("CREDITSCORE").cast("float")
)


spdf_drift.write.mode("overwrite").save_as_table("CUSTOMERS_TRAINING")


In [ ]:
# Load preprocessing pipeline from a file
session.file.get('@ML_STAGE/preprocessing_pipeline.joblib.gz', '/tmp')
pipeline_file = '/tmp/preprocessing_pipeline.joblib.gz'


preprocessing_pipeline = joblib.load(pipeline_file)

# Apply preprocessing
training_spdf = preprocessing_pipeline.fit(spdf_drift).transform(spdf_drift)
training_spdf = training_spdf.with_column(
"CREDITSCORE_SS", col("CREDITSCORE_SS").cast("float")
)

num_cols = ['EstimatedSalary', 'Balance', 'CreditScore','Age','Tenure','debttoincome']

cat_cols = ['HasCrCard', 'IsActiveMember', 'GEOGRAPHY','GENDER', 'NumOfProducts']
Target = ["EXITED"]

feature_names_input = [c for c in training_spdf.columns if c not in ["EXITED", "TRANSACTIONTIMESTAMP", "CUSTOMERID","ESTIMATEDSALARY", "BALANCE", "CREDITSCORE","AGE","TENURE","DEBTTOINCOME","GEOGRAPHY","GENDER","PREDICTED_CHURN","DATASET_TYPE"]]

output_label = ["PREDICTED_CHURN"]
# Initialize a XGBClassifier object with input, label, and output column names
model = XGBClassifier(
    input_cols=feature_names_input,
    label_cols=Target,
    output_cols=output_label
    
)

# Train the classifier model using the training set
_ = model.fit(training_spdf)


## Log the model as a new version V2 with the same model name QS_CustomerChurn_classifier

In [ ]:
from snowflake.ml.registry import Registry
from snowflake.ml.model import type_hints

reg = Registry(session=session)

MODEL_NAME = "QS_CustomerChurn_classifier"
MODEL_VERSION = "v2"

mv = reg.log_model(model,
                   model_name=MODEL_NAME,
                   version_name=MODEL_VERSION,
                   options={'relax_version': True},
                   task=type_hints.Task.TABULAR_BINARY_CLASSIFICATION)
reg.show_models()


## Create a model monitor on the new model version

In [ ]:
query = f"""
CREATE OR REPLACE MODEL MONITOR CHURN_MODEL_MONITOR_NEW
WITH
    MODEL=QS_CustomerChurn_classifier
    VERSION=v2
    FUNCTION=predict
    SOURCE=CUSTOMERS_EVAL
    BASELINE=CUSTOMERS_TRAINING
    TIMESTAMP_COLUMN=TRANSACTIONTIMESTAMP
    PREDICTION_CLASS_COLUMNS=(PREDICTED_CHURN)  
    ACTUAL_CLASS_COLUMNS=(EXITED)
    ID_COLUMNS=(CUSTOMERID)
    WAREHOUSE=ML_WH
    REFRESH_INTERVAL='1 min'
    AGGREGATION_WINDOW='1 day';
"""
session.sql(query).collect()

## Predict the churn on the new customer data with the retrained model

In [ ]:
status= inference('CUSTOMERS_EVAL','QS_CUSTOMERCHURN_CLASSIFIER', 'v2');

### Retrieve the statistical summaries of a feature, label, or model prediction from a monitored model over time.
🔹 Use Case: Helps analyze trends in model performance, feature behavior, and prediction distribution.Metric Name could be {‘COUNT’, ‘COUNT_NULL’}
### The granularity can be of any form ‘<num> {DAY, WEEK, MONTH, QUARTER, YEAR}’ or ALL or NULL


In [ ]:

SELECT * FROM 
TABLE(
MODEL_MONITOR_STAT_METRIC(
'CHURN_MODEL_MONITOR', 'COUNT', 'PREDICTED_CHURN', '1 DAY', TO_TIMESTAMP_TZ('2024-11-01'), TO_TIMESTAMP_TZ('2025-02-06'))
) as a
JOIN (SELECT * FROM 
TABLE(
MODEL_MONITOR_STAT_METRIC(
'CHURN_MODEL_MONITOR_NEW', 'COUNT', 'PREDICTED_CHURN', '1 DAY', TO_TIMESTAMP_TZ('2024-11-01'), TO_TIMESTAMP_TZ('2025-02-06'))
)) as b ON a.EVENT_TIMESTAMP = b.EVENT_TIMESTAMP;


### Compute drift metrics for a specified feature, label, or model prediction over a given time period. This helps detect changes in data distributions (feature drift) or prediction shifts (concept drift).

In [ ]:

SELECT * FROM TABLE(MODEL_MONITOR_DRIFT_METRIC(
'CHURN_MODEL_MONITOR_NEW', 'DIFFERENCE_OF_MEANS', 'PREDICTED_CHURN', '1 DAY', TO_TIMESTAMP_TZ('2025-02-01'), TO_TIMESTAMP_TZ('2025-02-04')))

### Purpose: Fetches performance metrics for a monitored model over a specified time range.
🔹 Use Case: It allows tracking of how the model's performance has evolved (e.g., accuracy drops or performance degradation).

In [ ]:
SELECT * FROM TABLE(MODEL_MONITOR_PERFORMANCE_METRIC(
'CHURN_MODEL_MONITOR_NEW', 'PRECISION', '1 DAY', TO_TIMESTAMP_TZ('2024-11-01'), TO_TIMESTAMP_TZ('2025-02-05')))

### Setting up Alerts

Setup Alerts to receive notification when a certain metric goes over threshold limit

In [ ]:
query=f'''CREATE or replace TABLE TEST_NOTIFICATION(
    notification varchar (100),
    created_at timestamp
);'''

session.sql(query).collect()

In [ ]:
CREATE OR REPLACE ALERT high_drift_alert
    WAREHOUSE = ML_WH
    SCHEDULE = '60 minutes'
    IF ( EXISTS (SELECT * FROM TABLE(MODEL_MONITOR_DRIFT_METRIC(
    'CHURN_MODEL_MONITOR', 'DIFFERENCE_OF_MEANS', 'PREDICTED_CHURN', '1 MONTH', TO_TIMESTAMP_TZ('2024-01-01'), TO_TIMESTAMP_TZ('2025-02-04')))))
    THEN
        INSERT INTO TEST_NOTIFICATION (notification, created_at) VALUES ('ALERT',(SELECT CURRENT_TIMESTAMP));

## End of Notebook